# Definición de Funciones para los Endpoints de la API

A continuación, se describen las funciones creadas para los endpoints que serán consumidos en nuestra API:

1. `PlayTimeGenre(genero)`: Esta función devuelve el año con la mayor cantidad de horas jugadas para el género especificado.

2. `UserForGenre(genero)`: La función `UserForGenre` devuelve el usuario que acumula más horas jugadas para el género proporcionado, junto con una lista de la acumulación de horas jugadas por año.

3. `UsersRecommend(año)`: Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. Estos juegos tienen recomendaciones positivas o neutrales y tienen la máxima calificación por parte de los usuarios.

4. `UsersNotRecommend(año)`: Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. Estos juegos tienen recomendaciones negativas y comentarios críticos por parte de los usuarios.

5. `sentiment_analysis(año)`: Según el año de lanzamiento, esta función devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentran categorizados con un análisis de sentimiento. La lista muestra la cantidad de reseñas positivas, negativas y neutrales para ese año.

Estas funciones son esenciales para el funcionamiento de nuestra API y se encargan de procesar las solicitudes entrantes y generar respuestas adecuadas.

In [1]:
import pandas as pd # Importare las librerias necesarias
import datetime

In [2]:
# Leere el CSV 'PlayTimeGenre_funcion'
play_genre= pd.read_csv('PlayTimeGenre_funcion.csv', low_memory=False)

In [3]:
def PlayTimeGenre(genres, play_genre):
    '''
    Esta función devuelve el año con la mayor 
    cantidad de horas jugadas para el género especificado.
    '''
    # Voy a filtrar el DataFrame para incluir solo el género específico
    df_filtered = play_genre[play_genre['genres'] == genres]

    if df_filtered.empty:
        return f"No se encontraron datos para el género {genres}"

    # Los agrupare por usuario y año de lanzamiento y suma las horas jugadas
    usuario_año_playtime = df_filtered.groupby(['item_id', 'release_date'])['playtime_forever'].sum().reset_index()

    # En esta linea se encuentra el usuario con más horas jugadas para el género dado
    max_usuario = usuario_año_playtime.groupby('item_id')['playtime_forever'].sum().idxmax()

    # Incluyo solo las filas correspondientes al usuario con más horas jugadas
    df_max_usuario = usuario_año_playtime[usuario_año_playtime['item_id'] == max_usuario]

    # Agrupo por año y sumo las horas jugadas para el usuario 
    max_usuario_año_playtime = df_max_usuario.groupby('release_date')['playtime_forever'].sum()

    # Convierto el resultado en un formato específico
    max_usuario_año_playtime_list = [{"Año": str(year), "Horas": hours} for year, hours in max_usuario_año_playtime.items()]

    return {
        f"Genero con más horas jugadas  {genres}": max_usuario,
        "Horas jugadas": max_usuario_año_playtime_list
    }

genres = "Action"
result = PlayTimeGenre(genres, play_genre)
result


{'Genero con más horas jugadas  Action': 730.0,
 'Horas jugadas': [{'Año': '2004-06-01', 'Horas': 269.0},
  {'Año': '2009-03-23', 'Horas': 2786.0},
  {'Año': '2011-03-22', 'Horas': 25355.0},
  {'Año': '2012-07-05', 'Horas': 4659.0},
  {'Año': '2012-08-30', 'Horas': 265.0},
  {'Año': '2013-10-01', 'Horas': 2327.0},
  {'Año': '2014-10-02', 'Horas': 74076.0},
  {'Año': '2014-11-26', 'Horas': 61193.0},
  {'Año': '2015-01-15', 'Horas': 57204.0},
  {'Año': '2015-09-11', 'Horas': 189556.0},
  {'Año': '2016-06-02', 'Horas': 36303.0},
  {'Año': '2017-10-17', 'Horas': 11180.0},
  {'Año': '2017-12-08', 'Horas': 584.0}]}

In [4]:
# Leere el CSV 'UserForGenre_funcion'
user_genre= pd.read_csv('UserForGenre_funcion.csv', low_memory=False)

In [5]:
def UserForGenre(genres, user_genre):
    '''Vuelvo a crear la función que me devolvera al usuario con mayor jugadas
    los años y horas'''
    
    # filtrare el DataFrame para incluir solo el género específico
    df_filtro2 = user_genre[user_genre['genres'] == genres]

    # Esta linea corrige los valores de 'release_date' que no siguen el formato esperado
    df_filtro2 = df_filtro2[df_filtro2['release_date'].str.match(r'\d{4}-\d{2}-\d{2}', na=False)]

    # Lo convierto 'release_date' a un objeto de fecha y hora válido
    df_filtro2['release_date'] = pd.to_datetime(df_filtro2['release_date'], format='%Y-%m-%d')

    # Voy a agrupar por usuario y año de lanzamiento y suma las horas jugadas
    usuario_año_playtime = df_filtro2.groupby(['user_id', df_filtro2['release_date'].dt.year])['playtime_forever'].sum().reset_index()

    # Aqui se encuentra el usuario con más horas jugadas para el género dado
    max_usuario = usuario_año_playtime.groupby('user_id')['playtime_forever'].sum().idxmax()

    # incluyo solo las filas correspondientes al usuario con más horas jugadas
    df_max_usuario = usuario_año_playtime[usuario_año_playtime['user_id'] == max_usuario]

    # Agrupo por año y sumo las horas jugadas para el usuario.
    max_usuario_año_playtime = df_max_usuario.groupby('release_date')['playtime_forever'].sum()

    # Convierto el resultado a una lista de diccionarios
    max_usuario_año_playtime_list = [{"Año": year, "Horas": hours} for year, hours in zip(max_usuario_año_playtime.index, max_usuario_año_playtime)]

    return {
        f"Usuario con más horas jugadas para {genres}": max_usuario,
        "Horas jugadas": max_usuario_año_playtime_list
    }

genres = "Action Indie"
result = UserForGenre(genres, user_genre)

result


{'Usuario con más horas jugadas para Action Indie': 'thequeenpanda',
 'Horas jugadas': [{'Año': 2015, 'Horas': 97.0},
  {'Año': 2016, 'Horas': 294923.0}]}

In [6]:
# Leere el CSV 'UsersRecommend_funcion' 
user_recommend= pd.read_csv('UsersRecommend_funcion.csv', low_memory=False)

In [7]:
def UsersRecommend(year):
    '''Devuelve los 3 juegos más recomendados por usuarios para el año dado por un usuario específico.'''
    
    # Filtra reseñas recomendadas para el año y el usuario específico
    filtered_reviews = user_recommend[(user_recommend['release_date'].str.contains(str(year), regex=False, na=False)) & (user_recommend['recommend'] == True)]

    # Contar la cantidad de reseñas por título de juego
    game_counts = filtered_reviews['title'].value_counts().reset_index()
    game_counts.columns = ['title', 'count']

    # Seleccionar los 3 juegos más recomendados
    top_games = game_counts.head(3)

    # Crear una lista de diccionarios con los juegos más recomendados
    top_3_games_list = [{"Puesto {}: {}".format(i+1, game): count} for i, (game, count) in enumerate(zip(top_games['title'], top_games['count']))]

    return top_3_games_list

year = 2013
result = UsersRecommend(year)
result

[{'Puesto 1: Trine  Complete Story': 1},
 {'Puesto 2: Angelica Weaver Catch Me When You Can': 1},
 {'Puesto 3: DmC Devil May Cry': 1}]

In [8]:
# Ahora leere el CSV 'UsersNotRecommend_funcion'
juegos_no_recom= pd.read_csv('UsersNotRecommend_funcion.csv', low_memory=False)

In [9]:
def juegosNoRecomendados(año):
    
    ''' Y esta funcion lo que hara es lo contrario de la anterior, 
        devolvera los juegos no recomendados por los usuarios'''
        
    # Filtrare el DataFrame para incluir solo el año y las reseñas no recomendadas
    filtrando_reviews = juegos_no_recom[(juegos_no_recom['release_date'].str.contains(str(año), regex=False, na=False)) & (juegos_no_recom['recommend'] == False)]

    # Agrupo por título del juego y cuenta la cantidad de reseñas no recomendadas
    game_counts = filtrando_reviews['title'].value_counts().reset_index()
    game_counts.columns = ['title', 'count']

    # Ordeno los juegos por la cantidad de no recomendaciones de manera descendente
    juegos_no = game_counts.head(3)

    # Convierto el resultado en un formato de lista de diccionarios
    menos_3_juegos_list = [{"Puesto {}: ".format(i+1) + game: count} for i, (game, count) in enumerate(zip(juegos_no['title'], juegos_no['count']))]

    return menos_3_juegos_list

año = 2017
result = juegosNoRecomendados(año)
result


[{'Puesto 1: RC Plane   Canyon Scenario': 1},
 {'Puesto 2: RC Plane   FB': 1},
 {'Puesto 3: Chronicles of Vinland': 1}]

- En esta función tuve algunas dificultades con la columna 'sentiment_analysis', asi es que la converti en formato 'int' antes estaba en 'float' y borre algunos NaN ya que mi funcion no corria y me daba error.

In [10]:
# Leo el CSV 'sentiment_analysis_funcion'
df_sentimiento_analisis= pd.read_csv('sentiment_analysis_funcion.csv', low_memory=False)

In [11]:
def sentiment_analysis(anio):
    '''
    Función que devuelve la cantidad de registros de reseñas de usuarios 
    categorizados con un análisis de sentimiento para un anio de lanzamiento específico. 
    '''
    
    # Filtrar las reseñas por el anio especificado
    df_filtrado = df_sentimiento_analisis[df_sentimiento_analisis['release_date'].str.startswith(str(anio))]

        # Contar la cantidad de registros para cada categoría de análisis de sentimiento
    sentiment_counts = df_filtrado['sentiment_analysis'].value_counts()

    # Crear un diccionario con los resultados
    result_dict = {"Negative": 0, "Neutral": 0, "Positive": 0}
    
    for index, count in sentiment_counts.items():
        if index == 0:
            result_dict["Negative"] = count
        elif index == 1:
            result_dict["Neutral"] = count
        elif index == 2:
            result_dict["Positive"] = count

    return result_dict

count_sentiment = sentiment_analysis(2005)
print(count_sentiment)


{'Negative': 5, 'Neutral': 5, 'Positive': 11}
